## Process txt files

In [ ]:
!pip install pandas


In [2]:
import codecs

In [ ]:
# must run
import xml.etree.ElementTree as xml_parser
import re
import tarfile
from tqdm.auto import tqdm
import json
import os,codecs
from time import sleep
import pandas as pd
import urllib.request as ur
from random import randint



# test download one file 
# !wget ftp://ftp.ncbi.nlm.nih.gov/pub/pmc/oa_package/5f/a6/PMC7102175.tar.gz

# must run
def get_id_list_by_year_season(year, season=1):

    '''
    reldate
    When reldate is set to an integer n, ELink returns only those items that have a date specified by datetype within the last n days.
    # "cerebral palsy"[MeSH Terms] OR ("cerebral"[All Fields] AND "palsy"[All Fields]) OR "cerebral palsy"[All Fields]
    https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=pubmed&term=cerebral+palsy[MeSH+Terms]+OR+cerebral+palsy[All+Fields]+AND+Randomized+Controlled+Trial[Publication+Type]

    '''
    #term1 = "(cerebral+palsy[MeSH+Terms]+OR+cerebral+palsy[All+Fields]+AND+Randomized+Controlled+Trial[Publication+Type])"
    #term2 = "(cerebral+palsy[MeSH+Terms]+OR+cerebral+palsy[All+Fields]+AND+randomized+controlled+trial[All+Fields])"
    #term3 = "(randomized+controlled+trial[All+Fields])"
    #term3 = "(Randomized+Controlled+Trial[Publication+Type])"
    term4 = "(Observational+Study[Publication+Type])"
    term5 = "(Observational+Study%2C+Veterinary[Publication+Type])"
    term6 = term4 + "||" + term5
    term7 = "(Case+Reports[Publication+Type])"
    term8 = "(ffrft[Filter])"
    term9 = "+AND+(humans[Filter])+AND+(english[Filter])"
    term10 = term7 + '+AND+' + term8 + term9
    baseURL = 'http://eutils.ncbi.nlm.nih.gov/entrez/eutils/'
    #filter=pubt.randomizedcontrolledtrial
    #Randomized Controlled Trial[Publication Type]

    reldate=None
    eutil = 'esearch.fcgi?'
    dbParam = 'db=pubmed'#db=pubmed, db=pmc
    usehistoryParam = '&usehistory=y'
    retmode = '&retmode=JSON'
    eutilfetch = 'efetch.fcgi?'
    fieldParam = '&field=title'
    termParam = '&term='+ str(term10)
    reldateParam = '&reldate='+ str(reldate)+'&datetype=edat'
    if season==1:
        mindate='&mindate='+str(year)+'/01/01'
        maxdate='&maxdate='+str(year)+'/03/31'
    elif season==2:
        mindate='&mindate='+str(year)+'/04/01'
        maxdate='&maxdate='+str(year)+'/06/30'
    elif season==3:
        mindate='&mindate='+str(year)+'/07/01'
        maxdate='&maxdate='+str(year)+'/09/30'
    elif season==4:
        mindate='&mindate='+str(year)+'/10/01'
        maxdate='&maxdate='+str(year)+'/12/31'
    else:
        mindate='&mindate='+str(year)+'/01/01'
        maxdate='&maxdate='+str(year)+'/12/31'

        
    #crdt='&crdt=2021'
    retmax = 1000000
    retmaxParam = '&retmax='+str(retmax)
    #retstart = 0
    if reldate is not None and isinstance(reldate, int):
        fullURL = baseURL+eutil+dbParam+fieldParam+termParam+usehistoryParam+retmode+retmaxParam+reldateParam
        print (fullURL)
    else:
        fullURL = baseURL+eutil+dbParam+fieldParam+termParam+usehistoryParam+retmode+retmaxParam+mindate+maxdate
        #print ("Invalid/Empty reldate intput (need to be an INTEGER). Collecting publications from all time ...")
        print(fullURL)

    # =====urllib.error.HTTPError: HTTP Error 400: Bad Request======== dealing with non-ascii url =====
    try:
        fetch = ur.urlopen(fullURL)
    except:
        fullURL = urllib.parse.quote(fullURL,safe=':/') # <- here
        fetch = ur.urlopen(fullURL)

    datam = json.loads(fetch.read().decode('utf-8')) 
    idlist = []
   
    try:
        count = datam['esearchresult']['count']
    except:
        count = None
    if count is not None:
        idlist = datam['esearchresult']['idlist']

    return count, idlist



for i in range(1,3):
  year = i+2010
  count, idlist = get_id_list_by_year_season(year, 2)
  print(f'year {year}: {count}')
  print(len(idlist))
  print(idlist[:20])
  sleep(0.2)


In [ ]:
def save_id_list_season(year_begin, year_end):
    #Yingcheng Update
    #API instructions: https://dataguide.nlm.nih.gov/eutilities/utilities.html
    year = year_begin
    total_count=0
    total_list=[]
    while year <= year_end:
        for season in range(1,5):
            sleep(0.5)
            count,idlist = get_id_list_by_year_season(year, season)
            print(year, count, len(idlist))
            total_count+=len(idlist)

            path="data/"
            os.makedirs(path, exist_ok=True)
            output_dir = os.path.join(os.getcwd(),path+"pmid_"+str(year)+".list")
            outfile = codecs.open(output_dir, "a+")
            for pmid in idlist:
                outfile.write(pmid+"\n")
                total_list.append(int(pmid))

        year += 1 
    print(total_count)
    print(len(set(total_list)))
    
save_id_list_season(1960, 2022)

In [6]:
path = 'data/'
pmidlist= codecs.open(path+'pmid_'+str(2020)+'.list').readlines()
length = len(pmidlist)
print(f'total pmids: {length}')

total pmids: 20453


## Process Text

In [8]:
!ls ../case-report-race-data/text_data/
comm_dir = '../case-report-race-data/text_data/comm/unziped'

non_comm_dir = '../case-report-race-data/text_data/non_comm/unziped'

other_dir = '../case-report-race-data/text_data/other/unziped'

!ls $other_dir

comm     non_comm other
PMC000xxxxxx PMC002xxxxxx PMC004xxxxxx PMC006xxxxxx PMC008xxxxxx
PMC001xxxxxx PMC003xxxxxx PMC005xxxxxx PMC007xxxxxx PMC009xxxxxx


In [9]:
import os, codecs
from tqdm.auto import tqdm
import pandas as pd

In [10]:
txt_files_dict = {}

duplicates_pmcs=[]

def read_pmc_files_dict(txt_files_dict, folder_dir):
    for root, dirs, files in os.walk(folder_dir):
        for afile in files:
            if afile.endswith(".txt"):
                txt_file_name = os.path.basename(afile).replace('.txt', '').strip()
                if txt_file_name in txt_files_dict:
#                     print(txt_file_name+" already exist!")
                    duplicates_pmcs.append(txt_file_name)
                else:
                    txt_files_dict[txt_file_name] = os.path.join(root, afile)
    return txt_files_dict


pmc_dict = read_pmc_files_dict(txt_files_dict, other_dir)
pmc_dict = read_pmc_files_dict(txt_files_dict, comm_dir)
pmc_dict = read_pmc_files_dict(txt_files_dict, non_comm_dir)

print(len(duplicates_pmcs))

179


In [11]:
print(len(pmc_dict))

4974288


In [12]:
def get_content_from_file(file_dir):
    f = open(file_dir, 'r', errors='replace') 
    lines = f.readlines()
#     print(lines)
    start_idx = 0
    end_idx = 0
    for idx, line in enumerate(lines):
        if line.startswith('==== Body'):
            start_idx = idx
        elif line.startswith('====') and not line.startswith('==== Front') and idx > start_idx:
            end_idx = idx
            break
        else:
            end_idx = len(lines)-1
#             print('error: '+ k)

#     print(f'{k}\t{start_idx}\t{end_idx}')
    return ' '.join(lines[start_idx+1:end_idx])

In [13]:
for k, v in tqdm(pmc_dict.items()):
    file_dir = v
#     lines = codecs.open(file_dir, encoding='utf-8').readlines() # read the file we downloaded for case reports in US.
    # print(lines)
    print(get_content_from_file(file_dir))
    break

  0%|          | 0/4974288 [00:00<?, ?it/s]

pmc Clinical Implications
 
 Compared with non–eosinophilic gastrotintestinal disease (EGID) coronavirus disease 2019 (COVID-19) positive patients, EGID COVID-19 positive individuals stayed longer in the hospital, yet had a lower hazard of in-patient mortality. This analysis suggests that EGID may provide a protective effect against severe COVID-19 outcomes.
 
 In the United States, the coronavirus disease 2019 (COVID-19) pandemic was the third leading cause of death in 2020.1 Peripheral eosinophilia is hypothesized to play a protective role in COVID-19.2 Yet, little is known about eosinophilic gastrotintestinal disease (EGID) and COVID-19 outcomes. Th2 mucosal responses of patients with EGID may protect against severe effects of COVID-19 by reducing viral entry into cells.3 , 4 We hypothesized that EGID would be protective against severe outcomes in COVID-19 infections. We reviewed administrative data from an extensive central medical system in the United States to identify all COVID-

In [14]:
def load_dict():
    id_mappings_ftp = 'ftp://ftp.ncbi.nlm.nih.gov/pub/pmc/PMC-ids.csv.gz'
    df = pd.read_csv(id_mappings_ftp, dtype=str)
  # df.head()
    keys = df['PMID'].to_list()
    keys = [str(x).strip() for x in keys]
    values = df['PMCID'].to_list()
    values = [y.strip() for y in values]
    dictionary = dict(zip(keys, values))
    return dictionary

def get_pmcid_from_pmid(pmid, dictionary):
    if dictionary is None or len(dictionary)==0:
        print('loading pmid to pmcid dict...')
        dictionary = load_dict()
    else:
        pmid = str(pmid).strip()
        if pmid in dictionary:
            return dictionary[str(pmid)]
        else:
            return None

dictionary = load_dict()

In [15]:
print(len(dictionary))

8060898


In [16]:
for k, v in dictionary.items():
    print(k,v)
    break
    
print(dictionary[str(11056682)])

11250746 PMC13900
PMC13914


In [17]:
found_total_count = 0

for year in range(1960,2023):
    path = 'data/'
#     year = 1960
    print(f'processing year: {year}')
    output_path = path+'haotest/'+str(year)+'/'
    os.makedirs(output_path, exist_ok=True)

    pmid_pmcid_no_found=codecs.open(os.path.join(output_path,"pmid_pmcid_no_found.list"),'w')

    pmid_pmcid_found_no_content=codecs.open(os.path.join(output_path,"pmid_pmcid_found_no_content.list"),'w')

    pmidlist= codecs.open(path+'pmid_'+str(year)+'.list').readlines()
    length = len(pmidlist)
    print(f'total pmids: {length}')

    content_pmids = []

    for pmid in tqdm(pmidlist):
        pmid = str(pmid)
        pmid = pmid.strip()
        pmcid = get_pmcid_from_pmid(pmid, dictionary)
        if pmcid is None:
    #         print(f'cannot find pmcid for pmid: {pmid}, skip')
            pmid_pmcid_no_found.write(str(pmid)+"\n")
            continue
        else:
            if pmcid in pmc_dict:
                file_dir = pmc_dict[pmcid]
                content_pmids.append(pmid)
                content = get_content_from_file(file_dir)
    #             print(content)
                output_dir = os.path.join(os.getcwd(),output_path+str(pmid)+".txt")
                outfile = codecs.open(output_dir, "w")
                outfile.write(content)
                outfile.close()
                found_total_count+=1
            else:
    #             print(f'no content for pmcid: {pmcid}')
                pmid_pmcid_no_found.write(str(pmid)+"\n")


    pmid_pmcid_found_no_content.close()           
    pmid_pmcid_no_found.close()

    print(f'total content pmids: {len(content_pmids)}')
print(f'total content pmids count:{found_total_count}')

processing year: 1960
total pmids: 190


  0%|          | 0/190 [00:00<?, ?it/s]

total content pmids: 2
processing year: 1961
total pmids: 245


  0%|          | 0/245 [00:00<?, ?it/s]

total content pmids: 1
processing year: 1962
total pmids: 179


...

total content pmids: 11512
processing year: 2021
total pmids: 17430


  0%|          | 0/17430 [00:00<?, ?it/s]

total content pmids: 11297
processing year: 2022
total pmids: 12437


  0%|          | 0/12437 [00:00<?, ?it/s]

total content pmids: 8572
total content pmids count:78582


## Extract race info

In [18]:
folder_dir = 'data/haotest/'
files_dict ={}
for root, dirs, files in os.walk(folder_dir):
        for afile in files:
            if afile.endswith(".txt"):
                txt_file_name = os.path.basename(afile).replace('.txt', '').strip()
                files_dict[txt_file_name] = os.path.join(root, afile)
                

print(len(files_dict))

78669


In [130]:
import re

subjects = 'girl|boy|patient|american|american|child|children|participants|girls|boys|patients|americans|male|males|female|females|participant|woman|women|man|men|subject|subjects|islander|islander|native|native|natives|mother|mothers|father|fathers|soldier|infants|infant'

sub_patterns = r'((?:\S+\s+){0,3}\b'+f'({subjects})'+r'\b\s*(?:\S+\b\s*){0,3})'

# sub_patterns = r'((?:\S+\s+){0,3}\b'+f'({subjects}))'

print(sub_patterns)

text_all = 'A 5-year-old man and turkish boy was admitted to the hospital with chest tightness and palpitations which persisted for 2 years.'

matches = re.findall(sub_patterns, text_all.lower())

print(matches)
for match in matches:
    print(match)


def get_subjects(text_all):
    matches = re.findall(sub_patterns, text_all.lower())
    results = []
    for match in matches:
        s = ''.join(match[0])
        s = re.sub("\s+", " ", s)
        results.append(s)
    return results


print(get_subjects(text_all))



((?:\S+\s+){0,3}\b(girl|boy|patient|american|american|child|children|participants|girls|boys|patients|americans|male|males|female|females|participant|woman|women|man|men|subject|subjects|islander|islander|native|native|natives|mother|mothers|father|fathers|soldier|infants|infant)\b\s*(?:\S+\b\s*){0,3})
[('a 5-year-old man and turkish boy ', 'man')]
('a 5-year-old man and turkish boy ', 'man')
['a 5-year-old man and turkish boy ']


In [321]:
black_races_terms = 'black|south african|north african|caribbean|east african|north african|south african|west African|african|ghanaian|akan'

# black_patterns = f'({black_races_terms})([\w\s]'+'{,10})'+f'({subjects})'

black_patterns = f'({black_races_terms})'+r'([\b\s*(?:\S+\b\s*)]{0,10}?\b)'+f'({subjects})'

text_all = 'A 65-year-old black woman and north african girl was admitted to the hospital with chest tightness and palpitations which persisted for 2\xa0years. The patient had a free past medical history, and denied any other relevant personal or family history. '

def get_black_races(text_all):
    matches = re.findall(black_patterns, text_all.lower())
    results = []
    for match in matches:
        results.append(''.join(match))
    return results


print(get_black_races(text_all))



['black woman', 'north african girl']


In [322]:
white_races_terms = 'white|portuguese|french|british|european|lebanese|syrian|russian|caucasian|german|polish|ukrainian|arab|italian|english|irish|roma|scottish|welsh|turkish|algerian|iranian|moroccan|saudi|greek|omani|finnish'

# white_patterns = f'({white_races_terms})([\w\s]'+'{,10})'+f'({subjects})'

white_patterns = f'({white_races_terms})'+r'([\b\s*(?:\S+\b\s*)]{0,10}?\b)'+f'({subjects})'
# print(white_patterns)

text_all = 'A 65-year-old greek man was admitted to the hospital with chest tightness and palpitations which persisted for 2\xa0years. The patient had a free past medical history, and denied any other relevant personal or family history. '

def get_white_races(text_all):
    matches = re.findall(white_patterns, text_all.lower())
    results = []
    for match in matches:
        results.append(''.join(match))
    return results


print(get_white_races(text_all))

['greek man']


In [319]:
asian_races_terms = 'bangladeshi|bengali|cambodian|chinese|indian|malaysian|pakistani|filipino|thai|cambodia|china|india|japan|japanese|korea|korean|malaysia|pakistan|philippine|thailand|vietnam|vietnamese|taiwanese|asian|sri lankan'

# asian_patterns = f'({asian_races_terms})([\w\s]'+'{,10})'+f'({subjects})'

asian_patterns = f'({asian_races_terms})'+r'([\b\s*(?:\S+\b\s*)]{0,10}?\b)'+f'({subjects})'

text_all = 'A 65-year-old taiwanese woman and a 30-year-old bangladeshi-bengali female was admitted to the hospital with chest tightness and palpitations which persisted for 2\xa0years. The patient had a free past medical history, and denied any other relevant personal or family history. '

def get_asian_races(text_all):
    matches = re.findall(asian_patterns, text_all.lower())
    results = []
    for match in matches:
        results.append(''.join(match))
    return results


print(get_asian_races(text_all))

# matches = re.findall(patterns, text_all.lower())

# for match in matches:
#     print(''.join(match))

['taiwanese woman', 'bangladeshi-bengali female']


In [289]:
latino_races_terms = 'latino|latina|latinx|latin|pashtun|hispanic|cuban|mexican|puerto rican|south american|central american|spanish|brazilian|brazil'

# latino_patterns = f'({latino_races_terms})([\w\s]'+'{,10})'+f'({subjects})'

latino_patterns = f'({latino_races_terms})'+r'([\b\s*(?:\S+\b\s*)]{0,10}?\b)'+f'({subjects})'


text_all = 'A 65-year-old brazilian woman was admitted to the hospital with chest tightness and palpitations which persisted for 2\xa0years. The patient had a free past medical history, and denied any other relevant personal or family history. '

def get_latino_races(text_all):
    matches = re.findall(latino_patterns, text_all.lower())
    results = []
    for match in matches:
        results.append(''.join(match))
    return results


print(get_latino_races(text_all))

# matches = re.findall(patterns, text_all.lower())

# for match in matches:
#     print(''.join(match))

['brazilian woman']


In [290]:
american_indian_terms = 'aboriginal|amerindian|indigenous|american indian'

american_indian_patterns = f'({american_indian_terms})'+r'([\b\s*(?:\S+\b\s*)]{0,10}?\b)'+f'({subjects})'

text_all = 'A 65-year-old american indian woman and two indigenous girls was admitted to the hospital with chest tightness and palpitations which persisted for 2\xa0years. The patient had a free past medical history, and denied any other relevant personal or family history. '

def get_ameindian_races(text_all):
    matches = re.findall(american_indian_patterns, text_all.lower())
    results = []
    for match in matches:
        results.append(''.join(match))
    return results


print(get_ameindian_races(text_all))

# matches = re.findall(patterns, text_all.lower())

# for match in matches:
#     print(''.join(match))

['american indian woman', 'indigenous girl']


In [284]:
# Native Hawaiian or Other Pacific Islander
# Hawaiian
# Guamanian
# Samoan

islander_terms = 'hawaiian|hawaii|samoan|guamanian'

islander_patterns = f'({islander_terms})'+r'([\b\s*(?:\S+\b\s*)]{0,10}?\b)'+f'({subjects})'

text_all = 'A 65-year-old american samoan woman was admitted to the hospital with chest tightness and palpitations which persisted for 2\xa0years. The patient had a free past medical history, and denied any other relevant personal or family history. '

def get_islander_races(text_all):
    matches = re.findall(islander_patterns, text_all.lower())
    results = []
    for match in matches:
        results.append(''.join(match))
    return results


print(get_islander_races(text_all))

# matches = re.findall(patterns, text_all.lower())

# for match in matches:
#     print(''.join(match))


['samoan woman']


In [323]:
def get_races_all(text_all):
    race_list = []
    race_type = []
    if len(get_black_races(text_all))>0:
        race_list.extend(get_black_races(text_all))
        race_type.append('black')
    if len(get_white_races(text_all))>0:
        race_list.extend(get_white_races(text_all))
        race_type.append('white')
    if len(get_ameindian_races(text_all))>0:
        race_list.extend(get_ameindian_races(text_all))
        race_type.append('ameindian')
    if len(get_latino_races(text_all))>0:
        race_list.extend(get_latino_races(text_all))
        race_type.append('latino')
    if len(get_asian_races(text_all))>0:
        race_list.extend(get_asian_races(text_all))
        race_type.append('asian')
    if len(get_islander_races(text_all))>0:
        race_list.extend(get_islander_races(text_all))
        race_type.append('islander')
    return list(set(race_list)), list(set(race_type))

text_all = 'A 65-year-old hawaiian woman and a turkish boy were admitted to the hospital with chest tightness and palpitations which persisted for 2\xa0years. The patient had a free past medical history, and denied any other relevant personal or family history. '
race_list, race_type = get_races_all(text_all)
print(race_list)
print(race_type)


print(get_subjects(text_all))

['turkish boy', 'hawaiian woman']
['white', 'islander']
['a 65-year-old hawaiian woman and a turkish ', 'boy were admitted to ', '2 years. the patient had a free ']


In [324]:
df_list =[]

for pmid, file_dir in tqdm(files_dict.items()):
#     print(k)
#     print(file_dir)
    no_race = False
    no_text = False
    race_info = None
    subject_info = None
    race_type = 'NA'
    f = open(file_dir, 'r', errors='replace') 
    lines = f.readlines()
#     print(lines)
    if len(lines)>0:
        text_all = ' '.join(lines)
        race_info, race_type = get_races_all(text_all)
        subject_info = get_subjects(text_all)
        if len(race_info)>0:
            print(race_info)  
        else:
#             print('no race info')
            no_race=True
#             if len(subject_info)>0:
#                 print(subject_info)
    else:
#         print('no text')
        no_text= True
        no_race = True
    adflist=[pmid, race_info, race_type, subject_info, no_race, no_text]
    df_list.append(adflist)
        

  0%|          | 0/78669 [00:00<?, ?it/s]

['roma always man']
['caucasian male', 'caucasian female', 'asian male', 'asian female']
['japanese male']
['china\tfemale', 'china\tmale']
['brazilian woman']
['japanese male']
['japanese female']
['hispanic female']
['white male']
['asian man', 'caucasian man']
['hispanic male']
['korea). the patient']
['moroccan male']
['white male']
['chinese woman']
['japanese man']
['caucasian male', 'caucasian woman', 'asian male', 'asian woman']
['caucasian woman', 'brazilian woman', 'asian woman']
['thailand. there, man']
['hispanic male']
['korean girl']
['polished the man']
['japanese woman']
['caucasian male', 'asian male']
['european patient', 'caucasian female', 'asian female']
['asian man', 'caucasian man']
['blacks, 30% of men', 'african american']
['japanese male']
['asian female']
['asian patient', 'caucasian patient', 'mexican patient']
['chinese man']
['african american']
['japanese patient']
['white female']
['chinese girl']
['japan, 61 patient', 'japan, 59 patient', 'japan, 64 pat

['caucasian female', 'asian female']
['asian patient']
['caucasian male', 'asian male']
['hispanic male']
['korean patient', 'moroccan patient']
['caucasian male', 'asian male']
['asian woman']
['black female']
['chinese female']
['asian male']
['caucasian woman', 'asian woman']
['japanese man']
['caucasian woman', 'asian woman']
['germany). the patient']
['chinese female']
['black man']
['arable to our patient']
['sri lankan male']
['turkish patient', 'chinese patient']
['indian male']
['asian man', 'caucasian man']
['german child']
['asian girl', 'caucasian girl']
['brazilian patient']
['chinese patient']
['chinese man']
['asian man', 'caucasian man', 'english male']
['white male']
['chinese female']
['caucasian woman', 'asian woman']
['japanese man']
['white american']
['african american']
['chinese man']
['asian\xa0male', 'caucasian\xa0male']
['china. the patient', 'chinese woman']
['chinese spg7 patient', 'italian patient', 'chinese patient']
['japanese male']
['germany girl', 'ge

['italian man']
['french man']
['moroccan man']
['caucasian woman', 'asian woman']
['arable, the patient']
['white woman']
['moroccan man']
['japanese patient', 'chinese man']
['caucasian female', 'asian female']
['caucasian female', 'asian female']
['germany). the patient']
['caucasian male', 'asian male']
['korean man']
['turkish patient']
['korean patient', 'chinese patient']
['arab male']
['brazilian man']
['polish woman']
['japanese woman']
['asian woman']
['asian woman', 'caucasian man', 'asian man', 'caucasian woman', 'african-american', 'hispanic woman']
['chinese child']
['french-american']
['japanese woman']
['european patient', 'japanese woman']
['caucasian male', 'asian male']
['indian female']
['italian patient']
['english-speaking woman']
['european american', 'african american']
['indian male', 'french guianese male', 'amerindian male']
['japanese patient']
['chinese male']
['italian patient']
['caucasian male', 'asian male']
['chinese woman']
['asian man', 'caucasian ma

['japanese man']
['korean male']
['japan\t43\tfemale', 'india\t17\tmale', 'china\t77\tmale', 'japan\t66\tmale', 'japan\t46\tfemale', 'germany\t16\tmale', 'japan\t57\tmale', 'japan\t76\tmale', 'korea\t46\tmale', 'china\t18\tfemale', 'china\t24\tfemale', 'india\t65\tmale', 'korea\t68\tfemale', 'china\t58\tfemale']
['moroccan newborn female', 'asian g1p1 mother', 'caucasian g1p1 mother']
['caucasian male', 'asian male']
['japanese woman']
['asian patient', 'caucasian patient']
['asian obese female', 'chinese female', 'chinese patient']
['caucasian male', 'asian male']
['filipino male']
['korean child', 'korea. the patient']
['indian female']
['italian boy']
['taiwanese man']
['black circles (female']
['chinese female']
['moroccan patient']
['asian patient', 'caucasian patient']
['iranian woman']
['brazil, where patient']
['germany). the patient']
['japanese man']
['white slavic male', 'lating the man']
['chinese man']
['caucasian male', 'asian male']
['japanese boy', 'japanese patient']
[

...

['hispanic man']
['japanese male']
['chinese girl', 'chinese boy', 'chinese patient']
['japanese man']
['white) girl', 'hispanic white) girl', 'european–american']
['japanese male']
['japanese man']
['asian man']
['asian patient']
['asian indian male']
['indian man']
['pakistani man']
['lating the man']
['korean female']
['european american', 'japanese man']
['caucasian woman', 'asian woman']
['japanese man']
['japanese-american', 'african-american']
['caucasian male', 'asian male']
['sri lankan female']
['japanese patient']
['latin american']
['japanese man']
['korean patient']
['turkish female']
['korean woman']
['japan are boy', 'japanese girl']
['indian baby girl']
['saudi diabetic female', 'saudi patient']
['syrian alawis woman']
['african–american']
['chinese female']
['african-american']
['chinese pregnant woman']
['japanese female']
['japanese female']
['japanese baby boy']
['arab male']
['brazil, in patient']
['japanese men', 'japanese patient']
['japanese girl', 'japanese wom

['arab man']
['african american']
['thai woman']
['korean woman', 'asian patient']
['japanese woman']
['white male']
['french-american']
['greek male']
['caucasian male', 'asian male']
['korean patient', 'japanese patient', 'asian infants', 'korean fhl2 patient']
['japanese man']
['indian patient', 'indian female']
['black african patient']
['chinese female']
['taiwanese patient', 'caucasian male', 'indian patient', 'french patient', 'asian patient', 'asian male']
['chinese male']
['japanese male']
['asian male']
['chinese man']
['korean man']
['hispanic male']
['caucasian male', 'asian male']
['arabia\n  \n  patient']
['arabia in a patient']
['saudi child', 'caucasian boy', 'asian boy']
['japanese man']
['asian man', 'caucasian man']
['chinese woman']
['white kurdish woman']
['japanese patient']
['iranian patient']
['european-american', 'chinese male']
['african mother']
['black (subject']
['white man']
['african american']
['white man']
['latin-american']
['taiwanese woman']
['korean

['arable to the patient']
['caucasian female', 'asian female']
['white woman']
['sri lankan man']
['european american', 'african american']
['european and american', 'russian woman']
['white male', 'white woman']
['caucasian women', 'asian women']
['japanese male']
['european descent girl']
['china; the patient']
['white male', 'white child']
['chinese man']
['latina female']
['japanese patient']
['european patient']
['chinese man']
['asian man', 'caucasian man']
['japanese man']
['lebanese patient']
['chinese woman']
['pakistani infant']
['white girl']
['african female']
['china). the patient']
['french canadian male']
['asian (dutch) male', 'asian man', 'caucasian man', 'caucasian (dutch) male']
['asian man', 'caucasian man']
['chinese woman', 'chinese man']
['asian man']
['japanese woman']
['latin-american']
['portuguese ps patient', 'portuguese female', 'chinese child']
['white woman']
['thai male', 'thai female']
['brazil men']
['asian men']
['latin american', 'italian man', 'ital

['chinese young girl']
['chinese girl', 'china. her mother']
['african-american']
['european american', 'white female', 'african american']
['chinese man']
['taiwanese man', 'asian man']
['lating the patient']
['caucasian male', 'asian male']
['white woman']
['korea. our patient']
['japanese male']
['caucasian female', 'asian female']
['asian patient', 'caucasian patient']
['asian male']
['caucasian male', 'asian male']
['white male']
['chinese patient']
['chinese girl']
['caucasian male', 'asian male']
['japanese man']
['caucasian woman', 'asian woman']
['japanese woman']
['thailand. the patient']
['caucasian female', 'asian female']
['japanese boy', 'spanish male']
['chinese male']
['chinese han boy']
['sri lankan man']
['black female']
['omani female']
['bengali man']
['chinese patient']
['caucasian woman', 'asian woman']
['japanese man']
['caucasian male', 'asian male']
['caucasian boy', 'asian boy']
['thai gd patient', 'chinese patient']
['african american']
['african-american']
[

['korean woman']
['korean woman']
['korea. man']
['roma in a child']
['lating having child']
['korean woman', 'korean patient', 'caucasian cadasil patient', 'asian cadasil patient']
['korea, patient']
['asian woman']
['korean man']
['japanese woman']
['white man']
['korea.\n  the patient', 'central american man', 'korean man']
['chinatown in man']
['korean\n  woman', 'korean woman', 'asians, marked female', 'european-american']
['korean boy']
['iranian woman']
['chinese patient']
['caucasian boy', 'asian boy']
['asian man']
['korean patient', 'korean\n  patient']
['thailand. for patient', 'caucasian woman', 'asian woman']
['caucasian woman', 'asian woman']
['korean woman']
['white men']
['japanese woman']
['korean woman']
['hispanic male']
['korea. the patient']
['korean woman', 'korean girl']
['saudi woman']
['korea and our patient']
['english and man']
['korean female']
['white male']
['asians. two patient']
['romas\n  in patient']
['white female']
['french patient']
['korean male']


In [325]:
df = pd.DataFrame(df_list, columns =['pmid', 'race_info', 'race_type', 'subject_info', 'no_race', 'no_text'])
df.head()

,pmid,race_info,race_type,subject_info,no_race,no_text
0,23775233,[],[],"[occur in younger women and has been , (cag). case the patient was a 34-year-old , woman who presented with , of her second child, 1b), and the patient’s systolic blood , because of the patient’s unstable hemodynamic , 1c), and the patient’s hemodynamic state , h later. the patient also complained of , was excluded. the patient’s postoperative course , dissections in our patient were secondary (iatrogenic, % of scad patients are reported to , be female, our case, the patient had a history , dissection. among younger patients, pathological studies of patients who died because , reported that all patients treated with initial , according to the patient’s clinical presentation, observed exclusively in women [1, ], women treated for primary , have described a patient with scad who , into account the patient’s condition and ]",True,False
1,24050515,[],[],"[reported in a patient who had an , and in a patient who had a , parahaemolyticus. a 50-year-old woman, and depression. the patient’s family owned , 12.5–15.5 g/dl for women, the icu, the patient received ampicillin/clavulanate and , specimen of a patient with septic shock , disease of the patient (including a coma , consolidations), and the patient rapidly improved after , agent of the patient’s disease, pathogen, especially in patients who have aspiration ]",True,False
2,36028867,[roma always man],[white],"[primarily detected in infants and children but , nonspecific, and symptomatic patients may present with , presentation a 65-year-old woman was admitted to , 2 years. the patient had a free , tumor removal, the patient was free of , follow-up period, this patient experienced no tumor , nonspecific, and symptomatic patients may present with , our case, the patient only presented with , 3-year follow-up, our patient had no recurrence , and treated the patient, obtained from the patient, obtained from the patient for the publication ]",False,False
3,36187048,[],[],"[majority of the children with bowlegs, of a four-year-old boy with infantile blount´s , disease. patient and observation information , of the patient, : a four-year-old boy presented with bowing , development of the child was age appropriate , therapeutic interventions: the child was then referred , and mobility. the child continued to receive , to which the child could perform his , obtained from the patient´s parents for , of legs in children, corrects as the child grows older, status of the child as well [5, is seen in children aged between 1-4 , age. clinically, the child has a variable , present. as the child walks a characteristic , by langenskiöld. the children belonging to langenskiöld , of one year. children with langenskiöld stage , glance of the child, alternative diagnoses. the child underwent a successful , disease are overweight child, can change a child´s life by , was involved in patient management, the initial manuscript, patient´s data collection]",True,False
4,23587009,"[caucasian male, caucasian female, asian male, asian female]","[white, asian]","[third of the patient presents with locally , one third of patients undergoing cytoreductive nephrectomy , cure to the patient and is usually , for poor prognosis patients according to mskcc , 24 months [10]. patients with mrcc who , agents. furthermore, in patients undergoing hemodialysis the , and toxicity in patients undergoing dialysis [11,12, we present 2 patients with bilateral rcc , a 50-year-old caucasian female with a medical , and urea the patient required renal replacement , september 2009 the patient complained of hematemesis , sunitinib administration the patient experienced left hemiparesis , 2009 did the patient resume therapy based , mtor inhibitor. the patient was on everolimus , because of the patient’s deteriorating performance , a 51-year-old caucasian male, stage t1n0. the patient required renal replacement , the resection the patient manifested a subacute , resi

In [326]:
df.shape

(78669, 6)

In [327]:
file_name = 'pmid_race_subject_all_v2.csv'
df.to_csv(file_name, index=False)

In [328]:
print(df[df['no_text']==True].shape)
df[df['no_text']==True].head(10)
# df.loc[df['no_text'] == True, 'no_race'] = True

(425, 6)


,pmid,race_info,race_type,subject_info,no_race,no_text
20862,3077055,None,NA,None,True,True
20866,3267363,None,NA,None,True,True
20867,3267367,None,NA,None,True,True
20869,3267366,None,NA,None,True,True
20871,3267364,None,NA,None,True,True
20872,3267358,None,NA,None,True,True
20873,3267359,None,NA,None,True,True
20874,3267365,None,NA,None,True,True
20879,3267355,None,NA,None,True,True
20881,3267357,None,NA,None,True,True


In [329]:
print(df[(df['no_race']==True) & (df['no_text']==False)].shape)

(60669, 6)


In [330]:
# print(df[df['race_type'].str.contains("white")])

df[df['race_type'].apply(lambda x: 'white' in x)].shape

(7409, 6)

In [331]:
# df[df['race_type'].str.contains("asian")]

df[df['race_type'].apply(lambda x: 'asian' in x)].shape

(11808, 6)

In [332]:
# df[df['race_type'].str.contains("black")]

df[df['race_type'].apply(lambda x: 'black' in x)].shape

(1538, 6)

In [333]:
# df[df['race_type'].str.contains("latino")]

df[df['race_type'].apply(lambda x: 'latino' in x)].shape

(1332, 6)

In [334]:
# df[df['race_type'].str.contains("ameindian")]

df[df['race_type'].apply(lambda x: 'ameindian' in x)].shape

(55, 6)

In [335]:
# df[df['race_type'].str.contains("islander")]

df[df['race_type'].apply(lambda x: 'islander' in x)].shape

(9, 6)

In [336]:
eval_df = df[(df['no_race']==True) & (df['no_text']==False)]

eval_df.shape

(60669, 6)

In [337]:
eval_df_sample = eval_df.sample(n=500, random_state=44)
eval_df_sample.shape

(500, 6)

In [338]:
eval_df_sample.head(10)

,pmid,race_info,race_type,subject_info,no_race,no_text
35277,31666823,[],[],"[rehabilitation program in patients suffering from ischemic , treatment regimen in patients with acute brain , 1 a 59-year-old female patient was brought , service intubated the patient who received bag-valve-mask , avf leads. the patient was admitted to , mechanical ventilation. the patient was unstable hemodynamically, day 12, the patient was taken off , and extubated. the patient stayed in the , neurological unit, the patient stayed for 17 , at discharge, the patient presented with improved , independent feeding. the patient walked independently with , 2 a 42-year-old female patient was admitted , was severe: the patient was unconscious, absent bilaterally. the patient was mechanically ventilated , day 7. the patient was transferred to , 3 a 65-year-old male patient was admitted , and hypertension. the patient was intubated in , emergency room. the patient arrived in the , was noticed. the patient was connected to , also administered. the patient was extubated on , cardiac function. the patient was intubated and , 33 days, the patient was transferred to , at discharge, the patient was conscious, spectacular recovery in patients with severe brain ]",True,False
64041,29517669,[],[],"[with hyperglycemia. elderly patients with type 2 , diabetes, women, report a 70-year-old male patient presented to , of dizziness. the patient denied diplopia, result. subsequently, the patient was admitted, of a 66-year-old woman with hyperosmolar hyperglycemic , of an elderly female patient with transient , of hemichorea in patients with diabetic ketoacidosis, . our patient demonstrated a remissive , ketotic hyperglycemia, no patient presented with delayed ]",True,False
72498,27057880,[],[],"[mainly affecting elder patients.1,3 pathological myopia , 17% of cnv patients.5 although these , occur, from which patients suffer for a , relatively contradicted on patients with heart or , both doctors and patients to some extent, imaging to adjust patients, by octa during patients, case series recruited patients with icnv or , from all included patients, criteria contained: (1). patients diagnosed with cnv , or icnv, (2). patients were treatment-naïve or , penetrating the retina. patients undertook a thorough , conducted on all patients, outcomes and follow-up patients were followed up , general characteristics of patients were listed in , eyes from 10 patients were included in , of them were females and the other , 4 (40%) were males, and 2 (20%) patients had obvious intraretinal , except 4 (40%) patients got only 1 , and 1 (10%) patient got 4 (once , (20%) of 10 patients had cnv recurred , of these 10 patients was demonstrated in , cmt. table 1 patient characteristics and demographics , information of included patients changes or best-corrected , bcva of included patients in each follow-up, bcva of 10 patients was (0.81 , 1 a 52-year-old female complaint of vision , of a 52-year-old woman diagnosed with cnv , images of the patient in figure 1, 2 a 47-year-old male patient was diagnosed , of a 47-year-old man with a history , (octa) of the patient from figure 3, imaging on cnv patients, primary examination when patients are suspected with , 4 of 10 patients had their cnv , injections and 1 patient got 4 injections , lesion. among recurrent patients, supply in normal patients, contrary to normal patients with adequate capillaries, choriocapillaris of amd patients in their study , be restricted to patients without rapid eye , images of incoordinate patients turned out to , we excluded these patients to achieve better , leakage. for pm patients, of cnv. second, patients with severe rpe , both examiners and patients, first, only 10 patients were included in , quite impossible for patients to move their , to acquire if patients are unable to ]",True,False
1697,36305023,[],[],"[2022 a 74-year-old woman was admitted to , a thyroiditis. the patient did not undergo , cytology (fnac). t

In [314]:
from ast import literal_eval

pattern_to_check = r'(young|old|month|year|day)'
def filter_string_list(astr_list):
    results = []
    for x in astr_list:
        matches = re.findall(pattern_to_check, x.lower())
        for match in matches:
            results.append(x.strip())
            break
#     results = [re.sub('\W+',' ', x) for x in results]
    if len(results) == 0 and len(astr_list)>0:
        results.append(astr_list[0])
    return ' || '.join(results)

astr_list = ['affect mainly young men.3,4 case report ', 'a 25 year-old man was hospitalized in ', 'was begun the patient had significant improvement', 'who described three patients with a triple ', 'reaches 60-70% of patients in turkey and ', 'observed in bd patients from northern europe ', 'half of all patients but is more ', 'severe among the male and the young', 'small fraction of patients indicates a bad ', 'bcs of our patient are due to ', 'authors treat these patients only with immunosuppressive ', 'of bleeding4. our patient showed initial response ', 'and complications of patients with bs']

print(filter_string_list(astr_list))

affect mainly young men.3,4 case report || a 25 year-old man was hospitalized in || severe among the male and the young


In [58]:
import pandas as pd

pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)


In [315]:

# eval_df_sample.subject_info = eval_df_sample.subject_info.apply(literal_eval)


eval_df_sample['subject_info_simple'] = eval_df_sample['subject_info'].apply(lambda x: filter_string_list(x))



In [309]:
eval_df_sample.columns

Index(['pmid', 'race_info', 'race_type', 'subject_info', 'no_race', 'no_text',
       'subject_info_simple'],
      dtype='object')

In [316]:
eval_df_sample[['pmid', 'race_info', 'race_type', 'no_race', 'no_text',
       'subject_info_simple']]

,pmid,race_info,race_type,no_race,no_text,subject_info_simple
67220,30107829,[],[],True,False,presentation a 54-year-old man with cutaneous plasmacytosis
75494,27247748,[],[],True,False,"2010, a 39-year-old man fractured both bones || in osteoporotic old patients"
1795,36012532,[],[],True,False,"or old age, female sex || present a young man with severe heart || presentation a 20-year-old man was diagnosed as"
51198,33994339,[],[],True,False,of a 39-year-old male physician
25615,29145318,[],[],True,False,report a 54-year-old man with a 3-month
53330,32590781,[],[],True,False,description a 57-year-old man was admitted to || description a 51-year-old female was admitted to || description a 58-year-old man was admitted to || postoperative day. the patient was asymptomatic and
2103,35819833,[],[],True,False,report a 10-year-old girl presented to an
42931,33389941,[],[],True,False,report a 9-year-old patient was referred to || disease in young patients
50557,34961499,[],[],True,False,", a 32-month-old boy || 2019, a 13-year-old girl || months after the patient stopped temozolomide treatment || month later, the patient received tumor resection"
27852,28894926,[],[],True,False,a previously well male


## PMID meta info

In [ ]:
!ls ../case-report-race-data/text_data/
comm_dir = '../case-report-race-data/text_data/comm/unziped'

non_comm_dir = '../case-report-race-data/text_data/non_comm/unziped'

other_dir = '../case-report-race-data/text_data/other/unziped'

!ls $other_dir

In [163]:
import xml.etree.ElementTree as xml_parser
import re
import tarfile
from tqdm.auto import tqdm
import json
import os,codecs
from time import sleep
import pandas as pd
import urllib.request as ur
from random import randint

In [164]:
def download_abstract_by_pmid(pmid, save_path):
    test_url = 'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=pubmed&id='+str(pmid)+'&retmode=XML'
    fetch = ur.urlopen(test_url)
    datam = fetch.read().decode('utf-8')  # xml of one article
    filename_xml = str(pmid)+'.xml'
    file_dir = os.path.join(save_path, filename_xml)
    xml = open(file_dir, "w")
    xml.write(datam)
    xml.close()


# In[6]:


# must run
# save pdfs 
# step 1: get each pmid for specified year
# step 2: get pmcid from a pmid
# step 3: get the download link for pmcid if it exists (meaning full text available on PMC free article repository) 
# step 4: download study pdf or tar files from the download link
# step 5: save pdf and rename
def save_xmls(year_begin, year_end):
    path = 'data/'
    if not os.path.exists(path):
        os.mkdir(path)

    year = year_begin

    while year <= year_end:
        print (year)
        pmidlist= codecs.open(path+'pmid_'+str(year)+'.list').readlines()
        length = len(pmidlist)
        pm_path = os.path.join(path+'xml/',str(year))
        
        print("dir for xml saved in", pm_path)
        os.makedirs(pm_path, exist_ok=True)

        exceptionlistfile=codecs.open(os.path.join(pm_path,"exceptionlist"),'w')

        for (index,id) in enumerate(tqdm(pmidlist, total=length, position=0, leave=True)):
            id = id.rstrip()
            filename_xml=id+".xml"

            # print(f'processing id: {id}')
            
            if os.path.exists(os.path.join(pm_path,filename_xml)):
                print ("existed, skipped")
                continue
            if re.search("^#",id):
                continue 
            if re.search("^pmid", id):
                continue
            if re.search("^\s*$",id):
                continue
            id = id.rstrip().split("\t")[0]

            if (index) % 1000 == 0:
                print(index,"finished...")
            sleep(0.3)

            pmid = id
            # print(f'processing pmid: {pmid}')
            for i in range(5):
                try:
                    download_abstract_by_pmid(pmid, pm_path)
                    break
                except Exception as e:
                    print(f"Retrying: {i}")
                    sleep(randint(1,10))
            else:
                print(f'extraction failed at {pmid}')
                exceptionlistfile.write(str(id)+"\n")
                continue

        print("Retrieved xml saved in", pm_path)
        year+=1


# In[ ]:


# save_xmls(2022, 2022)
# save_xmls(2021, 2021)
# save_xmls(1960, 1969)
save_xmls(1970, 1979)

1960
dir for xml saved in data/xml/1960


  0%|          | 0/190 [00:00<?, ?it/s]

0 finished...
Retrieved xml saved in data/xml/1960
1961
dir for xml saved in data/xml/1961


  0%|          | 0/245 [00:00<?, ?it/s]

0 finished...
Retrying: 0
Retrieved xml saved in data/xml/1961
1962
dir for xml saved in data/xml/1962


...

0 finished...
Retrying: 0
Retrying: 0
Retrying: 0
Retrieved xml saved in data/xml/1967
1968
dir for xml saved in data/xml/1968


  0%|          | 0/227 [00:00<?, ?it/s]

0 finished...
Retrying: 0
Retrying: 0
Retrieved xml saved in data/xml/1968
1969
dir for xml saved in data/xml/1969


  0%|          | 0/188 [00:00<?, ?it/s]

0 finished...
Retrieved xml saved in data/xml/1969


In [ ]:
import xml.etree.ElementTree as xml_parser
import re
import tarfile
from tqdm.auto import tqdm
import json
import os,codecs
from time import sleep
import pandas as pd
import urllib.request as ur
import pickle
from random import randint


In [ ]:
def parse_abstract_by_file_path(file_path):
    one_file = file_path
    try:
        with open(one_file, 'r') as f:
            datam = f.read()
    except Exception as e:
        print(e)
        return None
    
    datam = re.sub("<i>", "", datam)  # remove <i> in xml to avoid failure in parsing .text
    datam = re.sub("</i>", "", datam)
    datam = re.sub("<sup>", "", datam)  # remove <i> in xml to avoid failure in parsing .text
    datam = re.sub("</sup>", "", datam)
    datam = re.sub("<sub>", "", datam)  # remove <i> in xml to avoid failure in parsing .text
    datam = re.sub("</sub>", "", datam)
    datam = re.sub("<em>", "", datam)  # remove <i> in xml to avoid failure in parsing .text
    datam = re.sub("</em>", "", datam)
    datam = re.sub("<strong>", "", datam)  # remove <i> in xml to avoid failure in parsing .text
    datam = re.sub("</strong>", "", datam)
    datam = re.sub("<b>", "", datam)  # remove <b> in xml to avoid failure in parsing .text
    datam = re.sub("</b>", "", datam)

    xmldoc = xml_parser.fromstring(datam)
    
    PubmedArticle = xmldoc.find("PubmedArticle")
    abstract_text = ""
    title_text=""
    meta_data={}

    meta_data['pubdate'] = ""
    meta_data['source'] = ""
    meta_data['volume'] = ""
    meta_data['pages'] = ""
    meta_data['pubtype'] = []
    meta_data['authors'] = []
    meta_data['affiliations'] = []


    if PubmedArticle is not None:
        data = PubmedArticle.find("MedlineCitation")
        if data is not None:
            article = data.find("Article")
            if article is not None:
                title = article.find("ArticleTitle")
                #print("Title:",title.text)
                title_text = title.text
                abstract = article.find("Abstract")
                if abstract is not None:
                    for seg in abstract:
                        if seg.text:
                            if 'NlmCategory' in seg.attrib:
                                abstract_text = abstract_text + "\n" + seg.attrib['NlmCategory'] + " : " + seg.text
                            elif 'Label' in seg.attrib:
                                abstract_text = abstract_text + "\n" + seg.attrib['Label'] + " : " + seg.text
                            else:
                                abstract_text = abstract_text + "\n" + seg.text
                        elif 'Label' in seg.attrib and len(seg.attrib['Label'].split(':'))>1:
                            abstract_text = abstract_text + "\n" + seg.attrib['Label'].split(':')[0] + " : " + seg.attrib['Label'].split(':')[1]
                
                journal = article.find("Journal")
                if journal is not None:
                    if journal.find("Title") is not None:
                        meta_data['source_title'] = journal.find("Title").text
                    if journal.find("ISOAbbreviation") is not None:
                        meta_data['source'] = journal.find("ISOAbbreviation").text
                    if journal.find("JournalIssue").find('Volume') is not None:
                        meta_data['volume'] = journal.find("JournalIssue").find('Volume').text
                   
                Pagination = article.find("Pagination")
                if Pagination is not None:
                    meta_data['pages'] = Pagination.find("MedlinePgn").text 
    

                AuthorList = article.find('AuthorList')

                if AuthorList is not None:
                    for author in AuthorList.findall('Author'):
                        if author.find('Initials') is not None and author.find('LastName') is not None:
                            meta_data['authors'].append(author.find('Initials').text+'. '+author.find('LastName').text)
                        if author.find('AffiliationInfo') is not None:
                            meta_data['affiliations'].append(author.find('AffiliationInfo').find('Affiliation').text)
                PublicationTypeList = article.find('PublicationTypeList')
                if PublicationTypeList is not None:
                    for PublicationType in PublicationTypeList.findall('PublicationType'):
                        meta_data['pubtype'].append(PublicationType.text)
        
        saved_mesh_list = []
        mesh_list = data.find("MeshHeadingList")
        if mesh_list is not None:
            for mesh in mesh_list.findall('MeshHeading'):
                mesh_id = mesh.find('DescriptorName').attrib['UI']
                mesh_name = mesh.find('DescriptorName').text
                saved_mesh_list.append((mesh_id, mesh_name))
        meta_data['mesh_list'] = saved_mesh_list
        
        saved_kwd_list = []
        kwd_list = data.find("KeywordList")
        if kwd_list is not None:
            for kwd in kwd_list.findall('Keyword'):
                saved_kwd_list.append(kwd.text)
        meta_data['kwd_list'] = saved_kwd_list    
                    
        
        PubmedData = PubmedArticle.find('PubmedData')
        if PubmedData is not None:
            History = PubmedData.find('History')
            if History.find("PubMedPubDate") is not None:
                date = History.find("PubMedPubDate")
                Year=""
                Month=""
                Day=""
                if date.find('Year') is not None:
                    Year =date.find('Year').text
                if date.find('Month') is not None:
                    Month=date.find('Month').text+'/'
                if date.find('Day') is not None:
                    Day=date.find('Day').text+'/'
                
                meta_data['pubdate'] = Month + Day +Year
                
    meta_data['title'] = title_text
    #print (meta_data)

    #abstract_text = re.search("©|(copyright)","", abstract_text)
    abstract_text = re.sub("^\s+", "", abstract_text)

    return title_text, abstract_text, meta_data





In [ ]:
def get_all_files(rootdir):
    files_dirs = []
    for root, dirs, files in os.walk(rootdir):
        for name in files:
            if name.endswith((".xml")):
                full_path = os.path.join(root, name)
                files_dirs.append(full_path)
    return files_dirs

rootdir = 'data/'

files_with_dir = get_all_files(rootdir)
len(files_with_dir)

In [ ]:
afile_path = files_with_dir[12564]
print(afile_path)
parse_abstract_by_file_path(afile_path)

In [ ]:
meta_list = []
for file_dir in tqdm(files_with_dir):
    file_name = os.path.basename(file_dir)
    apmid = file_name.replace('.xml', '')
    title_text, abstract_text, meta_data = parse_abstract_by_file_path(file_dir)
    meta_data['pmid']= apmid
#     print(meta_data)
    meta_list.append(meta_data)
#     print(meta_list)

In [ ]:
df = pd.DataFrame(meta_list)
df.tail()

In [ ]:

df.to_csv('all_case_report_meta_with_affiliation.tsv',sep='\t',index=False)
